<a href="https://colab.research.google.com/github/sailorcosmos101/Deeplearning_KU_2023/blob/main/PINN_laminar_flow_1115.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import time
import matplotlib
import matplotlib.pyplot as plt
import pickle
import scipy.io
import random


In [2]:
import tensorflow as tf

In [3]:
class PINN_laminar_flow:
  def __init__(self,Collo,INLET,OUTLET,WALL,uv_layers,lb,ub,ExistModel=0,uvDir=''):
    # Count for callback function
    self.count=0

    # Bounds
    self.lb=lb
    self.ub=ub

    # Mat. properties
    self.rho=1.0
    self.mu=0.02

    # Collocation point
    self.x_c=Collo[:,0:1]
    self.y_c=Collo[:,1:2]

    self.x_INLET=INLET[:,0:1]
    self.y_INLET=INLET[:,1:2]
    self.u_INLET=INLET[:,2:3]
    self.v_INLET=INLET[:,3:4]

    self.x_OUTLET=OUTLET[:,0:1]
    self.y_OUTLET=OUTLET[:,1:2]

    self.x_WALL=WALL[:,0:1]
    self.y_WALL=WALL[:,1:2]

    self.uv_layers=uv_layers
    self.loss_rec=[]

    # Initialize NNs
    self.uv_weights, self.uv_biases = self.initialize_NN(self.uv_layers)

    # tf placeholders
    self.learning_rate = tf.placeholder(tf.float32, shape=[])
    self.x_tf = tf.placeholder(tf.float, shape=[None,self.x_c.shape[1]])
    self.y_tf = tf.placeholder(tf.float, shape=[None,self.y_c.shape[1]])
    self.x_WALL_tf = tf.placeholder(tf.float, shape=[None,self.x_WALL.shape[1]])
    self.y_WALL_tf = tf.placeholder(tf.float, shape=[None,self.y_WALL.shape[1]])
    self.x_INLET_tf = tf.placeholder(tf.float, shape=[None,self.x_INLET.shape[1]])
    self.y_INLET_tf = tf.placeholder(tf.float, shape=[None,self.y_INLET.shape[1]])
    self.u_INLET_tf = tf.placeholder(tf.float, shape=[None,self.u_INLET.shape[1]])
    self.v_INLET_tf = tf.placeholder(tf.float, shape=[None,self.v_INLET.shape[1]])
    self.x_OUTLET_tf = tf.placeholder(tf.float, shape=[None,self.x_OUTLET.shape[1]])
    self.y_OUTLET_tf = tf.placeholder(tf.float, shape=[None,self.y_OUTLET.shape[1]])
    self.x_c_tf = tf.placeholder(tf.float, shape=[None,self.x_c.shape[1]])
    self.y_c_tf = tf.placeholder(tf.float, shape=[None,self.y_c.shape[1]])

    # tf graphs
    self.u_pred, self.v_pred, self.p_pred, _, _, _ = self.net_uv(self.x_Tf, self.y_tf)
    self.f_pred_u, self.f_pred_v, self.f_pred_s11, self.f_pred_s22, self.f_pred_s12, self.f_pred_p = self.net_f(self.x_c_tf, self.y_c_tf)
    self.u_WALL_pred, self.v_WALL_pred, _, _, _, _ = self.net_uv(self.x_WALL_tf, self.y_WALL_tf)
    self.u_INLET_pred, self.v_INLET_pred, _, _, _, _ = self.net_uv(self.x_INLET_tf, self.y_INLET_tf)
    _, _, self.p_OUTLET_pred, _, _, _ = self.net_uv(self.x_OUTLET_tf, self.y_OUTLET_tf)

    self.loss_f = tf.reduce_mean(tf.square(self.f_pred_u)) + tf.reduce_mean(tf.square(self.f_pred_v)) + tf.reduce_mean(tf.square(self.f_pred_s11)) + tf.reduce_mean(tf.square(self.f_pred_s22)) + tf.reduce_mean(tf.square(self.f_pred_s12)) + tf.reduce_mean(tf.square(self.f_pred_p))
    self.loss_WALL = tf.reduce_mean(tf.square(self.u_WALL_pred)) + tf.reduce_mean(tf.square(self.v_WALL_pred))
    self.loss_INLET = tf.reduce_mean(tf.square(self.u_INLET_pred-self.u_INLET_Tf)) + tf.reduce_mean(tf.square(self.v_INLET_pred-self.v_INLET_tf))
    self.loss_OUTLET = tf.reduce_mean(tf.square(self.p_OUTLET_pred))
    self.loss = self.loss_f + 2*(self.loss_WALL + self.loss_INLET + self.loss_OUTLET)

    # Optimizer for solution
    self.optimizer = tf.contrib.opt.ScipyOptimizerInterface(self.loss,
                                                                var_list=self.uv_weights + self.uv_biases,
                                                                method='L-BFGS-B',
                                                                options={'maxiter': 100000,
                                                                         'maxfun': 100000,
                                                                         'maxcor': 50,
                                                                         'maxls': 50,
                                                                         'ftol': 1*np.finfo(float).eps})
    self.optimizer_Adam = tf.train.AdamOptimizer(learning_rate = self.learning_rate)
    self.train_op_Adam = self.optimizer_Adam.minimize(self.loss,var_list=self.uv_weights + self.uv_biases)

    # tf session
    self.sees = tf.Session(config=tf.ConfigProto(allow_soft_placement=True, log_device_placement=True))
    init=tf.global_variables_initializer()
    self.sees.run(init)

  def initialize_NN(self,layers):
    weights = []
    biases = []
    num_layers = len(layers)
    for l in range(0,num_layers - 1):
      W = self.xavier_init(size=[layers[l], layers[l+1]])
      b= = tf.Variable(tf.zeros([1, layers[1+l]], dtype=tf.float32), dtype=tf.float32)
      weights.append(W)
      biases.append(b)
    return weights, biases

  def xavier_init(self, size):
    in_dim = size[0]
    out_dim = size[1]
    xavier_staddev = np.sqrt(2 / (in_dim + out_dim))
    return tf.Variable(tf.truncated_normal([in_dim, out_dim], stddev=xavier_stddev, dtype=tf.float32), dtype=tf.float32)

  def save_NN(self, fileDir):

  def load_NN(self, fileDir, layers):

  def neural_net(self,X,weights,biases):
      num_layers = len(weights)+1
      H = X

      for l in range(0, num_layers - 2):
        W = weights[l]
        b = biases[l]
        H = tf.tanh(tf.add(tf.matmul(H,W),b))
      W = weights[-1]
      b = biases[-1]
      Y = tf.add(tf.matul(H, W),b)
      return Y

  def net_uv(self, x, y):
    psips = self.neural_net(tf.concat([x, y],1), self.uv_weights, self.uv_biases)
    psi = psips[:,0:1]
    p = psips[:,1:2]
    s11 = psips[:, 2:3]
    s22 = psips[:, 3:4]
    s12 = psips[:, 4:5]
    u = tf.gradients(psi, y)[0]
    v = -tf.gradients(psi, x)[0]
    return u, v, p, s11, s22, s12

  def net_f(self, x, y):
    rho = self.rho
    mu = self.mu
    u, v, p, s11, s22, s12 = self.net_uv(x,y)

    s11_1
    s12_2
    s22_2
    s12_1

    u_x
    u_y


  def callback(self,loss):
    self.count = self.count+1
    self.loss_rec.append(loss)
    print('{}th iterations, Loss: {}'.format(self.count,loss))

  def train(self, iter, learning_rate):
    tf_dict = {self.x_c_tf: self.x_c,
               self.y_c_tf: self.y_c,
               self.x_WALL_tf: self.x_WALL,
               self.y_WALL_tf: self.y_WALL,
               self.x_INLET_tf: self}
    loss_WALL = []
    loss_f = []
    loss_INLET = []
    loss_OUTLET = []

    for it in range(iter):
      self.sess.run()

  # bfgs???
  def train_bfgs(self):


  def predict(self,x_star,y_star):

  # Loss function
  def getloss(self):

    tf_dict

    loss_f = self.sess.run
    loss_uv = self.sess.run



IndentationError: ignored

In [ ]:
def Preprocess(dir=):
  X = data['x']
  Y = data['y']

  x_star = X.flatten()[:,None]
  y_star = Y.flatten()[:,None]

  return x_star, y_star

In [ ]:
def Postprocess():


In [ ]:
if __name__=="__main__":
  # Domain bounds
  lb =
  up =

  # Network configuration

  # Wall = [x,y]
  wall_up =
  wall_lw =

  # INLET = [x,y]
  U_max =

  # Collocation point for equation residual

  # Visualize the collocation points

  # Train from scratch

  # Load trained neural network

  # Save neural network

  # Save loss history

  # Load fluent result

  # Get mixed-form PINN prediction

